In [57]:
## testing the cross corelation and convolution of the spectra

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


class model_class(nn.Module):
    def __init__(self):
        super().__init__()
        self.Layer1 = nn.Linear(2,500)               
        self.Layer2 = nn.Linear(500,1000)              
        self.Layer3 = nn.Linear(1000,2500)
        self.Layer4 = nn.Linear(2500,4500)
        self.Layer5 = nn.Linear(4500,5000)              
        self.Layer6 = nn.Linear(5000,6000)             
        
    def forward(self, x):
        out = F.relu(self.Layer1(x))
        out = F.relu(self.Layer2(out))
        out = F.relu(self.Layer3(out))
        out = F.relu(self.Layer4(out))
        out = F.relu(self.Layer5(out))
        out = self.Layer6(out) 
        return(out)
    
def predict_Imu(teff, logg, model, Path, band = 'K'):
    param_input = torch.from_numpy(
        np.array([teff/1000, logg], dtype='float32'))

    checkpoint = torch.load(Path)
    model.load_state_dict(checkpoint['model_state_dict'])

    # multiplying by 100000, as it was divided by the same while training
    output_Imu = model(param_input)*100000
    Imu = output_Imu.detach().numpy()
    if band.upper() == 'H':
        Imu = np.reshape(Imu, (12, 500)).T * 1e5
    elif band.upper() =='K':
        Imu = np.reshape(Imu, (12, 500)).T * 1e5
    elif band.upper() == 'R':
    # Imu = np.swapaxes(Imu_lambda,0,1)

    Imunorm = Imu/np.expand_dims(Imu[:, 0], 1)  # normalising

    # Imunorm = np.flip(Imunorm, axis=1)

    return Imu, Imunorm

In [59]:
Path_model_MYSTIC = '/home/nebrahimkutt/SPICA/NN_MYSTIC/final_model_MYSTIC_LR_500wlenT.pth'

mastorad = 4.8481368110953594e-09     # 
teff = 5320.  # float(input("Enter the effective temperature: "))
teff_std = 45
logg = 4.48  # float(input("Enter the surface gravity (log g): "))
logg_std = 0.05
theta = 1.  # float(input("Enter the angular diameter (estimate): "))
angdiam_rad = theta*mastorad

In [60]:
"""Loading the fits file for the stars"""
from astropy.io import fits


def read_oifits(fits_path):
    hdu = fits.open(fits_path)
    wavelength = hdu['OI_WAVELENGTH'].data['EFF_WAVE']
    star_vis2 = hdu['OI_VIS2'].data['VIS2DATA']
    star_vis2err = hdu['OI_VIS2'].data['VIS2ERR']
    ucoord = hdu['OI_VIS2'].data['UCOORD']
    vcoord = hdu['OI_VIS2'].data['VCOORD']
    phi = hdu['OI_T3'].data['T3PHI']
    hdu.close()

    basecoord = np.sqrt(ucoord**2+vcoord**2)
    spatial_freq = basecoord[:, None]/wavelength
    return wavelength, star_vis2, star_vis2err, ucoord, vcoord, phi, basecoord, spatial_freq

fits_path = '/home/nebrahimkutt/SPICA/MYSTIC_DATA_VRAD/MYSTIC_L2.2024Aug18.HD_10700.MIRCX_IDL.XYZ.AVG10m.oifits'

wavelength, star_vis2, star_vis2err, ucoord, vcoord, phi, basecoord, spatial_freq = read_oifits(fits_path)


In [107]:
wavelength.shape

(21,)

In [72]:
import matplotlib.pyplot as plt 
%matplotlib qt

plt.figure()
plt.scatter(spatial_freq, star_vis2, s = 5, c='k')

In [62]:
wavelength_model_MIRCX = np.array([1.0007e-06, 1.0022e-06, 1.0036e-06, 1.0051e-06, 1.0066e-06,
       1.0080e-06, 1.0095e-06, 1.0109e-06, 1.0124e-06, 1.0139e-06,
       1.0153e-06, 1.0168e-06, 1.0182e-06, 1.0197e-06, 1.0212e-06,
       1.0226e-06, 1.0241e-06, 1.0255e-06, 1.0270e-06, 1.0285e-06,
       1.0299e-06, 1.0314e-06, 1.0328e-06, 1.0343e-06, 1.0358e-06,
       1.0372e-06, 1.0387e-06, 1.0401e-06, 1.0416e-06, 1.0431e-06,
       1.0445e-06, 1.0460e-06, 1.0474e-06, 1.0489e-06, 1.0504e-06,
       1.0518e-06, 1.0533e-06, 1.0547e-06, 1.0562e-06, 1.0577e-06,
       1.0591e-06, 1.0606e-06, 1.0620e-06, 1.0635e-06, 1.0650e-06,
       1.0664e-06, 1.0679e-06, 1.0693e-06, 1.0708e-06, 1.0723e-06,
       1.0737e-06, 1.0752e-06, 1.0766e-06, 1.0781e-06, 1.0796e-06,
       1.0810e-06, 1.0825e-06, 1.0839e-06, 1.0854e-06, 1.0869e-06,
       1.0883e-06, 1.0898e-06, 1.0912e-06, 1.0927e-06, 1.0942e-06,
       1.0956e-06, 1.0971e-06, 1.0985e-06, 1.1000e-06, 1.1015e-06,
       1.1029e-06, 1.1044e-06, 1.1058e-06, 1.1073e-06, 1.1088e-06,
       1.1102e-06, 1.1117e-06, 1.1131e-06, 1.1146e-06, 1.1161e-06,
       1.1175e-06, 1.1190e-06, 1.1204e-06, 1.1219e-06, 1.1234e-06,
       1.1248e-06, 1.1263e-06, 1.1277e-06, 1.1292e-06, 1.1307e-06,
       1.1321e-06, 1.1336e-06, 1.1350e-06, 1.1365e-06, 1.1380e-06,
       1.1394e-06, 1.1409e-06, 1.1423e-06, 1.1438e-06, 1.1453e-06,
       1.1467e-06, 1.1482e-06, 1.1496e-06, 1.1511e-06, 1.1526e-06,
       1.1540e-06, 1.1555e-06, 1.1569e-06, 1.1584e-06, 1.1599e-06,
       1.1613e-06, 1.1628e-06, 1.1642e-06, 1.1657e-06, 1.1672e-06,
       1.1686e-06, 1.1701e-06, 1.1715e-06, 1.1730e-06, 1.1745e-06,
       1.1759e-06, 1.1774e-06, 1.1788e-06, 1.1803e-06, 1.1818e-06,
       1.1832e-06, 1.1847e-06, 1.1861e-06, 1.1876e-06, 1.1891e-06,
       1.1905e-06, 1.1920e-06, 1.1934e-06, 1.1949e-06, 1.1964e-06,
       1.1978e-06, 1.1993e-06, 1.2007e-06, 1.2022e-06, 1.2037e-06,
       1.2051e-06, 1.2066e-06, 1.2080e-06, 1.2095e-06, 1.2110e-06,
       1.2124e-06, 1.2139e-06, 1.2153e-06, 1.2168e-06, 1.2183e-06,
       1.2197e-06, 1.2212e-06, 1.2226e-06, 1.2241e-06, 1.2256e-06,
       1.2270e-06, 1.2285e-06, 1.2299e-06, 1.2314e-06, 1.2329e-06,
       1.2343e-06, 1.2358e-06, 1.2372e-06, 1.2387e-06, 1.2402e-06,
       1.2416e-06, 1.2431e-06, 1.2445e-06, 1.2460e-06, 1.2475e-06,
       1.2489e-06, 1.2504e-06, 1.2518e-06, 1.2533e-06, 1.2548e-06,
       1.2562e-06, 1.2577e-06, 1.2591e-06, 1.2606e-06, 1.2621e-06,
       1.2635e-06, 1.2650e-06, 1.2664e-06, 1.2679e-06, 1.2694e-06,
       1.2708e-06, 1.2723e-06, 1.2737e-06, 1.2752e-06, 1.2767e-06,
       1.2781e-06, 1.2796e-06, 1.2810e-06, 1.2825e-06, 1.2840e-06,
       1.2854e-06, 1.2869e-06, 1.2883e-06, 1.2898e-06, 1.2913e-06,
       1.2927e-06, 1.2942e-06, 1.2956e-06, 1.2971e-06, 1.2986e-06,
       1.3000e-06, 1.3015e-06, 1.3029e-06, 1.3044e-06, 1.3059e-06,
       1.3073e-06, 1.3088e-06, 1.3102e-06, 1.3117e-06, 1.3132e-06,
       1.3146e-06, 1.3161e-06, 1.3175e-06, 1.3190e-06, 1.3205e-06,
       1.3219e-06, 1.3234e-06, 1.3248e-06, 1.3263e-06, 1.3278e-06,
       1.3292e-06, 1.3307e-06, 1.3321e-06, 1.3336e-06, 1.3351e-06,
       1.3365e-06, 1.3380e-06, 1.3394e-06, 1.3409e-06, 1.3424e-06,
       1.3438e-06, 1.3453e-06, 1.3467e-06, 1.3482e-06, 1.3497e-06,
       1.3511e-06, 1.3526e-06, 1.3540e-06, 1.3555e-06, 1.3570e-06,
       1.3584e-06, 1.3599e-06, 1.3613e-06, 1.3628e-06, 1.3643e-06,
       1.3657e-06, 1.3672e-06, 1.3686e-06, 1.3701e-06, 1.3716e-06,
       1.3730e-06, 1.3745e-06, 1.3759e-06, 1.3774e-06, 1.3789e-06,
       1.3803e-06, 1.3818e-06, 1.3832e-06, 1.3847e-06, 1.3862e-06,
       1.3876e-06, 1.3891e-06, 1.3905e-06, 1.3920e-06, 1.3935e-06,
       1.3949e-06, 1.3964e-06, 1.3978e-06, 1.3993e-06, 1.4008e-06,
       1.4022e-06, 1.4037e-06, 1.4051e-06, 1.4066e-06, 1.4081e-06,
       1.4095e-06, 1.4110e-06, 1.4124e-06, 1.4139e-06, 1.4154e-06,
       1.4168e-06, 1.4183e-06, 1.4197e-06, 1.4212e-06, 1.4227e-06,
       1.4241e-06, 1.4256e-06, 1.4270e-06, 1.4285e-06, 1.4300e-06,
       1.4314e-06, 1.4329e-06, 1.4343e-06, 1.4358e-06, 1.4373e-06,
       1.4387e-06, 1.4402e-06, 1.4416e-06, 1.4431e-06, 1.4446e-06,
       1.4460e-06, 1.4475e-06, 1.4489e-06, 1.4504e-06, 1.4519e-06,
       1.4533e-06, 1.4548e-06, 1.4562e-06, 1.4577e-06, 1.4592e-06,
       1.4606e-06, 1.4621e-06, 1.4635e-06, 1.4650e-06, 1.4665e-06,
       1.4679e-06, 1.4694e-06, 1.4708e-06, 1.4723e-06, 1.4738e-06,
       1.4752e-06, 1.4767e-06, 1.4781e-06, 1.4796e-06, 1.4811e-06,
       1.4825e-06, 1.4840e-06, 1.4854e-06, 1.4869e-06, 1.4884e-06,
       1.4898e-06, 1.4913e-06, 1.4927e-06, 1.4942e-06, 1.4957e-06,
       1.4971e-06, 1.4986e-06, 1.5000e-06, 1.5015e-06, 1.5030e-06,
       1.5044e-06, 1.5059e-06, 1.5073e-06, 1.5088e-06, 1.5103e-06,
       1.5117e-06, 1.5132e-06, 1.5146e-06, 1.5161e-06, 1.5176e-06,
       1.5190e-06, 1.5205e-06, 1.5219e-06, 1.5234e-06, 1.5249e-06,
       1.5263e-06, 1.5278e-06, 1.5292e-06, 1.5307e-06, 1.5322e-06,
       1.5336e-06, 1.5351e-06, 1.5365e-06, 1.5380e-06, 1.5395e-06,
       1.5409e-06, 1.5424e-06, 1.5438e-06, 1.5453e-06, 1.5468e-06,
       1.5482e-06, 1.5497e-06, 1.5511e-06, 1.5526e-06, 1.5541e-06,
       1.5555e-06, 1.5570e-06, 1.5584e-06, 1.5599e-06, 1.5614e-06,
       1.5628e-06, 1.5643e-06, 1.5657e-06, 1.5672e-06, 1.5687e-06,
       1.5701e-06, 1.5716e-06, 1.5730e-06, 1.5745e-06, 1.5760e-06,
       1.5774e-06, 1.5789e-06, 1.5803e-06, 1.5818e-06, 1.5833e-06,
       1.5847e-06, 1.5862e-06, 1.5876e-06, 1.5891e-06, 1.5906e-06,
       1.5920e-06, 1.5935e-06, 1.5949e-06, 1.5964e-06, 1.5979e-06,
       1.5993e-06, 1.6008e-06, 1.6022e-06, 1.6037e-06, 1.6052e-06,
       1.6066e-06, 1.6081e-06, 1.6095e-06, 1.6110e-06, 1.6125e-06,
       1.6139e-06, 1.6154e-06, 1.6168e-06, 1.6183e-06, 1.6198e-06,
       1.6212e-06, 1.6227e-06, 1.6241e-06, 1.6256e-06, 1.6271e-06,
       1.6285e-06, 1.6300e-06, 1.6314e-06, 1.6329e-06, 1.6344e-06,
       1.6358e-06, 1.6373e-06, 1.6387e-06, 1.6402e-06, 1.6417e-06,
       1.6431e-06, 1.6446e-06, 1.6460e-06, 1.6475e-06, 1.6490e-06,
       1.6504e-06, 1.6519e-06, 1.6533e-06, 1.6548e-06, 1.6563e-06,
       1.6577e-06, 1.6592e-06, 1.6606e-06, 1.6621e-06, 1.6636e-06,
       1.6650e-06, 1.6665e-06, 1.6679e-06, 1.6694e-06, 1.6709e-06,
       1.6723e-06, 1.6738e-06, 1.6752e-06, 1.6767e-06, 1.6782e-06,
       1.6796e-06, 1.6811e-06, 1.6825e-06, 1.6840e-06, 1.6855e-06,
       1.6869e-06, 1.6884e-06, 1.6898e-06, 1.6913e-06, 1.6928e-06,
       1.6942e-06, 1.6957e-06, 1.6971e-06, 1.6986e-06, 1.7001e-06,
       1.7015e-06, 1.7030e-06, 1.7044e-06, 1.7059e-06, 1.7074e-06,
       1.7088e-06, 1.7103e-06, 1.7117e-06, 1.7132e-06, 1.7147e-06,
       1.7161e-06, 1.7176e-06, 1.7190e-06, 1.7205e-06, 1.7220e-06,
       1.7234e-06, 1.7249e-06, 1.7263e-06, 1.7278e-06, 1.7293e-06])

wavelength_model_MYSTIC = np.array([1.9504e-06, 1.9513e-06, 1.9522e-06, 1.9531e-06, 1.9540e-06,
       1.9549e-06, 1.9558e-06, 1.9567e-06, 1.9576e-06, 1.9585e-06,
       1.9594e-06, 1.9603e-06, 1.9612e-06, 1.9621e-06, 1.9630e-06,
       1.9639e-06, 1.9648e-06, 1.9657e-06, 1.9666e-06, 1.9675e-06,
       1.9684e-06, 1.9693e-06, 1.9702e-06, 1.9711e-06, 1.9720e-06,
       1.9729e-06, 1.9738e-06, 1.9747e-06, 1.9756e-06, 1.9765e-06,
       1.9774e-06, 1.9783e-06, 1.9792e-06, 1.9801e-06, 1.9810e-06,
       1.9819e-06, 1.9828e-06, 1.9837e-06, 1.9846e-06, 1.9855e-06,
       1.9864e-06, 1.9873e-06, 1.9882e-06, 1.9891e-06, 1.9900e-06,
       1.9909e-06, 1.9918e-06, 1.9927e-06, 1.9936e-06, 1.9945e-06,
       1.9954e-06, 1.9963e-06, 1.9972e-06, 1.9981e-06, 1.9990e-06,
       1.9999e-06, 2.0008e-06, 2.0017e-06, 2.0026e-06, 2.0035e-06,
       2.0044e-06, 2.0053e-06, 2.0062e-06, 2.0071e-06, 2.0080e-06,
       2.0089e-06, 2.0098e-06, 2.0107e-06, 2.0116e-06, 2.0125e-06,
       2.0134e-06, 2.0143e-06, 2.0152e-06, 2.0161e-06, 2.0170e-06,
       2.0179e-06, 2.0188e-06, 2.0197e-06, 2.0206e-06, 2.0215e-06,
       2.0224e-06, 2.0233e-06, 2.0242e-06, 2.0251e-06, 2.0260e-06,
       2.0269e-06, 2.0278e-06, 2.0287e-06, 2.0296e-06, 2.0305e-06,
       2.0314e-06, 2.0323e-06, 2.0332e-06, 2.0341e-06, 2.0350e-06,
       2.0359e-06, 2.0368e-06, 2.0377e-06, 2.0386e-06, 2.0395e-06,
       2.0404e-06, 2.0413e-06, 2.0422e-06, 2.0431e-06, 2.0440e-06,
       2.0449e-06, 2.0458e-06, 2.0467e-06, 2.0476e-06, 2.0485e-06,
       2.0494e-06, 2.0503e-06, 2.0512e-06, 2.0521e-06, 2.0530e-06,
       2.0539e-06, 2.0548e-06, 2.0557e-06, 2.0566e-06, 2.0575e-06,
       2.0584e-06, 2.0593e-06, 2.0602e-06, 2.0611e-06, 2.0620e-06,
       2.0629e-06, 2.0638e-06, 2.0647e-06, 2.0656e-06, 2.0665e-06,
       2.0674e-06, 2.0683e-06, 2.0692e-06, 2.0701e-06, 2.0710e-06,
       2.0719e-06, 2.0728e-06, 2.0737e-06, 2.0746e-06, 2.0755e-06,
       2.0764e-06, 2.0773e-06, 2.0782e-06, 2.0791e-06, 2.0800e-06,
       2.0809e-06, 2.0818e-06, 2.0827e-06, 2.0836e-06, 2.0845e-06,
       2.0854e-06, 2.0863e-06, 2.0872e-06, 2.0881e-06, 2.0890e-06,
       2.0899e-06, 2.0908e-06, 2.0917e-06, 2.0926e-06, 2.0935e-06,
       2.0944e-06, 2.0953e-06, 2.0962e-06, 2.0971e-06, 2.0980e-06,
       2.0989e-06, 2.0998e-06, 2.1007e-06, 2.1016e-06, 2.1025e-06,
       2.1034e-06, 2.1043e-06, 2.1052e-06, 2.1061e-06, 2.1070e-06,
       2.1079e-06, 2.1088e-06, 2.1097e-06, 2.1106e-06, 2.1115e-06,
       2.1124e-06, 2.1133e-06, 2.1142e-06, 2.1151e-06, 2.1160e-06,
       2.1169e-06, 2.1178e-06, 2.1187e-06, 2.1196e-06, 2.1205e-06,
       2.1214e-06, 2.1223e-06, 2.1232e-06, 2.1241e-06, 2.1250e-06,
       2.1259e-06, 2.1268e-06, 2.1277e-06, 2.1286e-06, 2.1295e-06,
       2.1304e-06, 2.1313e-06, 2.1322e-06, 2.1331e-06, 2.1340e-06,
       2.1349e-06, 2.1358e-06, 2.1367e-06, 2.1376e-06, 2.1385e-06,
       2.1394e-06, 2.1403e-06, 2.1412e-06, 2.1421e-06, 2.1430e-06,
       2.1439e-06, 2.1448e-06, 2.1457e-06, 2.1466e-06, 2.1475e-06,
       2.1484e-06, 2.1493e-06, 2.1502e-06, 2.1511e-06, 2.1520e-06,
       2.1529e-06, 2.1538e-06, 2.1547e-06, 2.1556e-06, 2.1565e-06,
       2.1574e-06, 2.1583e-06, 2.1592e-06, 2.1601e-06, 2.1610e-06,
       2.1619e-06, 2.1628e-06, 2.1637e-06, 2.1646e-06, 2.1655e-06,
       2.1664e-06, 2.1673e-06, 2.1682e-06, 2.1691e-06, 2.1700e-06,
       2.1709e-06, 2.1718e-06, 2.1727e-06, 2.1736e-06, 2.1745e-06,
       2.1754e-06, 2.1763e-06, 2.1772e-06, 2.1781e-06, 2.1790e-06,
       2.1799e-06, 2.1808e-06, 2.1817e-06, 2.1826e-06, 2.1835e-06,
       2.1844e-06, 2.1853e-06, 2.1862e-06, 2.1871e-06, 2.1880e-06,
       2.1889e-06, 2.1898e-06, 2.1907e-06, 2.1916e-06, 2.1925e-06,
       2.1934e-06, 2.1943e-06, 2.1952e-06, 2.1961e-06, 2.1970e-06,
       2.1979e-06, 2.1988e-06, 2.1997e-06, 2.2006e-06, 2.2015e-06,
       2.2024e-06, 2.2033e-06, 2.2042e-06, 2.2051e-06, 2.2060e-06,
       2.2069e-06, 2.2078e-06, 2.2087e-06, 2.2096e-06, 2.2105e-06,
       2.2114e-06, 2.2123e-06, 2.2132e-06, 2.2141e-06, 2.2150e-06,
       2.2159e-06, 2.2168e-06, 2.2177e-06, 2.2186e-06, 2.2195e-06,
       2.2204e-06, 2.2213e-06, 2.2222e-06, 2.2231e-06, 2.2240e-06,
       2.2249e-06, 2.2258e-06, 2.2267e-06, 2.2276e-06, 2.2285e-06,
       2.2294e-06, 2.2303e-06, 2.2312e-06, 2.2321e-06, 2.2330e-06,
       2.2339e-06, 2.2348e-06, 2.2357e-06, 2.2366e-06, 2.2375e-06,
       2.2384e-06, 2.2393e-06, 2.2402e-06, 2.2411e-06, 2.2420e-06,
       2.2429e-06, 2.2438e-06, 2.2447e-06, 2.2456e-06, 2.2465e-06,
       2.2474e-06, 2.2483e-06, 2.2492e-06, 2.2501e-06, 2.2510e-06,
       2.2519e-06, 2.2528e-06, 2.2537e-06, 2.2546e-06, 2.2555e-06,
       2.2564e-06, 2.2573e-06, 2.2582e-06, 2.2591e-06, 2.2600e-06,
       2.2609e-06, 2.2618e-06, 2.2627e-06, 2.2636e-06, 2.2645e-06,
       2.2654e-06, 2.2663e-06, 2.2672e-06, 2.2681e-06, 2.2690e-06,
       2.2699e-06, 2.2708e-06, 2.2717e-06, 2.2726e-06, 2.2735e-06,
       2.2744e-06, 2.2753e-06, 2.2762e-06, 2.2771e-06, 2.2780e-06,
       2.2789e-06, 2.2798e-06, 2.2807e-06, 2.2816e-06, 2.2825e-06,
       2.2834e-06, 2.2843e-06, 2.2852e-06, 2.2861e-06, 2.2870e-06,
       2.2879e-06, 2.2888e-06, 2.2897e-06, 2.2906e-06, 2.2915e-06,
       2.2924e-06, 2.2933e-06, 2.2942e-06, 2.2951e-06, 2.2960e-06,
       2.2969e-06, 2.2978e-06, 2.2987e-06, 2.2996e-06, 2.3005e-06,
       2.3014e-06, 2.3023e-06, 2.3032e-06, 2.3041e-06, 2.3050e-06,
       2.3059e-06, 2.3068e-06, 2.3077e-06, 2.3086e-06, 2.3095e-06,
       2.3104e-06, 2.3113e-06, 2.3122e-06, 2.3131e-06, 2.3140e-06,
       2.3149e-06, 2.3158e-06, 2.3167e-06, 2.3176e-06, 2.3185e-06,
       2.3194e-06, 2.3203e-06, 2.3212e-06, 2.3221e-06, 2.3230e-06,
       2.3239e-06, 2.3248e-06, 2.3257e-06, 2.3266e-06, 2.3275e-06,
       2.3284e-06, 2.3293e-06, 2.3302e-06, 2.3311e-06, 2.3320e-06,
       2.3329e-06, 2.3338e-06, 2.3347e-06, 2.3356e-06, 2.3365e-06,
       2.3374e-06, 2.3383e-06, 2.3392e-06, 2.3401e-06, 2.3410e-06,
       2.3419e-06, 2.3428e-06, 2.3437e-06, 2.3446e-06, 2.3455e-06,
       2.3464e-06, 2.3473e-06, 2.3482e-06, 2.3491e-06, 2.3500e-06,
       2.3509e-06, 2.3518e-06, 2.3527e-06, 2.3536e-06, 2.3545e-06,
       2.3554e-06, 2.3563e-06, 2.3572e-06, 2.3581e-06, 2.3590e-06,
       2.3599e-06, 2.3608e-06, 2.3617e-06, 2.3626e-06, 2.3635e-06,
       2.3644e-06, 2.3653e-06, 2.3662e-06, 2.3671e-06, 2.3680e-06,
       2.3689e-06, 2.3698e-06, 2.3707e-06, 2.3716e-06, 2.3725e-06,
       2.3734e-06, 2.3743e-06, 2.3752e-06, 2.3761e-06, 2.3770e-06,
       2.3779e-06, 2.3788e-06, 2.3797e-06, 2.3806e-06, 2.3815e-06,
       2.3824e-06, 2.3833e-06, 2.3842e-06, 2.3851e-06, 2.3860e-06,
       2.3869e-06, 2.3878e-06, 2.3887e-06, 2.3896e-06, 2.3905e-06,
       2.3914e-06, 2.3923e-06, 2.3932e-06, 2.3941e-06, 2.3950e-06,
       2.3959e-06, 2.3968e-06, 2.3977e-06, 2.3986e-06, 2.3995e-06])

Imu, Imunorm = predict_Imu(teff, logg, model= model_class(), Path = Path_model_MYSTIC, inst= 'MYSTIC')


/tmp/ipykernel_239007/2297761934.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(Path)


In [108]:
import matplotlib.patches as mpatches
plt.figure()
plt.plot(wavelength_model_MYSTIC, Imunorm, c = 'b', label = 'MYSTIC model')
plt.xlabel('wavelength')
plt.ylabel('Intensity')
# plt.legend()


Imu.shape
# wavelength_model.shape

(500, 12)

In [126]:
import numpy as np
from scipy.interpolate import interp1d
from PyAstronomy.pyasl import instrBroadGaussFast

resolution = 10000
Imu_convolved = []
new_wavelengths = None


for profile in Imunorm.T:
    result = instrBroadGaussFast(wavelength_model_MYSTIC, profile, resolution, edgeHandling="firstlast", equid=True, fullout=True)
    Imu_convolved.append(result[0])
    new_wavelengths = result[1]
    print(new_wavelengths)
Imu_convolved = np.array(Imu_convolved)

Imu_interp = np.array([interp1d(wavelength_model_MYSTIC, profile, kind='cubic', bounds_error=False, fill_value="extrapolate")(wavelength) for profile in Imu_convolved ])

2.1749500000000003e-10
2.1749500000000003e-10
2.1749500000000003e-10
2.1749500000000003e-10
2.1749500000000003e-10
2.1749500000000003e-10
2.1749500000000003e-10
2.1749500000000003e-10
2.1749500000000003e-10
2.1749500000000003e-10
2.1749500000000003e-10
2.1749500000000003e-10


In [161]:
Imu_interp.shape

(12, 21)

In [ ]:
channel_response = np.ones_like(wavelength)  # Uniform response
channel_response[(wavelength < wavelength.min()) | (wavelength > wavelength.max())] = 0  # Zero outside range


In [ ]:
from scipy.interpolate import interp1d
from scipy.integrate import quad
import numpy as np

# Ensure wavelength_target and Imu_interp are properly defined
Imu_function_list = [
    interp1d(wavelength, Imu_interp[i, :], kind='cubic', bounds_error=False, fill_value=0)
    for i in range(Imu_interp.shape[0])
]

channel_function = interp1d(wavelength, channel_response, kind='cubic', bounds_error=False, fill_value=0)


In [ ]:

def integrand(wavelength, imu_func):
    return imu_func(wavelength) * channel_function(wavelength)


In [ ]:
wavelength_min, wavelength_max = wavelength.min(), wavelength.max()

integrated_spectra = []
for imu_func in Imu_function_list:
    integrated_flux, error = quad(integrand, wavelength_min, wavelength_max, args=(imu_func,))
    integrated_spectra.append(integrated_flux)

# Convert to numpy array
integrated_spectra = np.array(integrated_spectra)

print(integrated_spectra.shape)  # Should be (12,)


In [ ]:
integrated_spectra

In [109]:
# plt.figure()
plt.plot(wavelength, Imu_interp.T, 'rx', label = 'MYSTIC interpolated')

from matplotlib.lines import Line2D

lines = Line2D([0], [0], color= 'blue', linewidth=1, linestyle='-')
xpoint = Line2D([0],[0], color='r', marker='x', linestyle='None')
labels = ['MYSTIC trained model', 'MYSTIC interpolated']

plt.legend([lines, xpoint], labels)

In [66]:
wavelength

array([2.3883745e-06, 2.3714297e-06, 2.3526422e-06, 2.3336261e-06,
       2.3143814e-06, 2.2949084e-06, 2.2752067e-06, 2.2552765e-06,
       2.2351178e-06, 2.2147306e-06, 2.1941148e-06, 2.1732706e-06,
       2.1521976e-06, 2.1308963e-06, 2.1093665e-06, 2.0876080e-06,
       2.0656212e-06, 2.0434056e-06, 2.0209618e-06, 1.9982892e-06,
       1.9834290e-06], dtype='>f4')

In [67]:
wavelength_model

array([1.0007e-06, 1.0022e-06, 1.0036e-06, 1.0051e-06, 1.0066e-06,
       1.0080e-06, 1.0095e-06, 1.0109e-06, 1.0124e-06, 1.0139e-06,
       1.0153e-06, 1.0168e-06, 1.0182e-06, 1.0197e-06, 1.0212e-06,
       1.0226e-06, 1.0241e-06, 1.0255e-06, 1.0270e-06, 1.0285e-06,
       1.0299e-06, 1.0314e-06, 1.0328e-06, 1.0343e-06, 1.0358e-06,
       1.0372e-06, 1.0387e-06, 1.0401e-06, 1.0416e-06, 1.0431e-06,
       1.0445e-06, 1.0460e-06, 1.0474e-06, 1.0489e-06, 1.0504e-06,
       1.0518e-06, 1.0533e-06, 1.0547e-06, 1.0562e-06, 1.0577e-06,
       1.0591e-06, 1.0606e-06, 1.0620e-06, 1.0635e-06, 1.0650e-06,
       1.0664e-06, 1.0679e-06, 1.0693e-06, 1.0708e-06, 1.0723e-06,
       1.0737e-06, 1.0752e-06, 1.0766e-06, 1.0781e-06, 1.0796e-06,
       1.0810e-06, 1.0825e-06, 1.0839e-06, 1.0854e-06, 1.0869e-06,
       1.0883e-06, 1.0898e-06, 1.0912e-06, 1.0927e-06, 1.0942e-06,
       1.0956e-06, 1.0971e-06, 1.0985e-06, 1.1000e-06, 1.1015e-06,
       1.1029e-06, 1.1044e-06, 1.1058e-06, 1.1073e-06, 1.1088e